In [101]:
import requests
import json
import pandas as pd

Acquiring Authorization Token

In [102]:
dict_ = {'client_id':'bh10ZpWky92tUnAljz3GIw',
        'client_secret' : 'hhG5pRxOM7rZGwH99mglPIf3w-wshw',
        'state':'123abcxyz',
        'redirect_uri':'https://google.com'
        }

In [103]:
url = 'https://www.reddit.com/api/v1/authorize'
params_ = {'client_id':dict_['client_id'],
           'response_type':'code',
           'state' : dict_['state'],
           'redirect_uri':dict_['redirect_uri'],
           'duration':'temporary',
           'scope': "read mysubreddits"}
headers = {'User-agent':'Venugopal Bhatia'}
token = requests.get(url,params = params_,headers = headers)
token.url

'https://www.reddit.com/login/?dest=https%3A%2F%2Fwww.reddit.com%2Fapi%2Fv1%2Fauthorize%3Fclient_id%3Dbh10ZpWky92tUnAljz3GIw%26response_type%3Dcode%26state%3D123abcxyz%26redirect_uri%3Dhttps%253A%252F%252Fgoogle.com%26duration%3Dtemporary%26scope%3Dread%2Bmysubreddits'

In [104]:
dict_['ret_redirect_uri'] = 'https://www.google.com/?state=123abcxyz&code=KhxkBkvJKyAy11QjbxPJs-m_tWGudg#_'

In [105]:
raw_vals = dict_['ret_redirect_uri'].split('?')[1].split("&")
url_params = {}
for val in raw_vals:
    key_val = val.split("=")
    url_params[key_val[0]] = key_val[1]
url_params['code'] = url_params['code'][:-2]
url_params

{'state': '123abcxyz', 'code': 'KhxkBkvJKyAy11QjbxPJs-m_tWGudg'}

Retrieving our access token

In [106]:
data_ = {'grant_type' : 'authorization_code',
         'code' : url_params['code'],
         'redirect_uri': dict_['redirect_uri']}

auth_ = (dict_['client_id'],dict_['client_secret'])

headers_ = {'User-agent':'Venugopal Bhatia'}

access_token = requests.post('https://www.reddit.com/api/v1/access_token',
                            auth=auth_,
                            data = data_,headers = headers_)
access_token_details = access_token.json()

In [107]:
access_token_details

{'access_token': '1324288346383-j3JqHp-lq6OAOTSpE8LNoK1c0VvAiQ',
 'token_type': 'bearer',
 'expires_in': 3600,
 'scope': 'read mysubreddits'}

Making requests with access token

In [37]:
Oauth_url_latest_posts = 'https://oauth.reddit.com/hot'
access_token = "bearer " + access_token_details['access_token']
headers = {'User-agent':'Venugopal_Bhatia',
           'authorization' : access_token}
response = requests.get(Oauth_url_latest_posts,headers = headers)
response

<Response [200]>

In [108]:
Oauth_user_subreddits = "https://oauth.reddit.com/subreddits/mine/subscriber"
access_token = "bearer " + access_token_details['access_token']
headers = {'User-agent':'Venugopal_Bhatia',
           'authorization' : access_token}
response = requests.get(Oauth_user_subreddits,headers = headers)
response

<Response [200]>

In [109]:
subreddit_data = json.loads(response.text)

In [110]:
subreddit_data

{'kind': 'Listing',
 'data': {'after': None,
  'dist': 4,
  'modhash': None,
  'geo_filter': '',
  'children': [{'kind': 't5',
    'data': {'user_flair_background_color': None,
     'submit_text_html': '&lt;!-- SC_OFF --&gt;&lt;div class="md"&gt;&lt;p&gt;&lt;strong&gt;Full text of rules can be found in the sidebar. Not reading the rules or adhering to &lt;a href="https://www.reddit.com/wiki/reddiquette"&gt;global reddiquette&lt;/a&gt; can result in removal of content.&lt;/strong&gt;&lt;/p&gt;\n\n&lt;p&gt;TL;DR&lt;/p&gt;\n\n&lt;blockquote&gt;\n&lt;ol&gt;\n&lt;li&gt;Flag all NSFW posts.&lt;/li&gt;\n&lt;li&gt;Clarify subject of post and do a bit of research.&lt;/li&gt;\n&lt;li&gt;No video or images, post a link in your text comment&lt;/li&gt;\n&lt;li&gt;No meme self/link-posts&lt;/li&gt;\n&lt;li&gt;Don&amp;#39;t harass other users.&lt;/li&gt;\n&lt;li&gt;Refrain from links/URLs to sites that require a login. BGlobe permitted.&lt;/li&gt;\n&lt;li&gt;No self/link posts that constitute adverti

In [111]:
subreddit_data['data']['children'][0]

{'kind': 't5',
 'data': {'user_flair_background_color': None,
  'submit_text_html': '&lt;!-- SC_OFF --&gt;&lt;div class="md"&gt;&lt;p&gt;&lt;strong&gt;Full text of rules can be found in the sidebar. Not reading the rules or adhering to &lt;a href="https://www.reddit.com/wiki/reddiquette"&gt;global reddiquette&lt;/a&gt; can result in removal of content.&lt;/strong&gt;&lt;/p&gt;\n\n&lt;p&gt;TL;DR&lt;/p&gt;\n\n&lt;blockquote&gt;\n&lt;ol&gt;\n&lt;li&gt;Flag all NSFW posts.&lt;/li&gt;\n&lt;li&gt;Clarify subject of post and do a bit of research.&lt;/li&gt;\n&lt;li&gt;No video or images, post a link in your text comment&lt;/li&gt;\n&lt;li&gt;No meme self/link-posts&lt;/li&gt;\n&lt;li&gt;Don&amp;#39;t harass other users.&lt;/li&gt;\n&lt;li&gt;Refrain from links/URLs to sites that require a login. BGlobe permitted.&lt;/li&gt;\n&lt;li&gt;No self/link posts that constitute advertising (for definition, see sidebar). &lt;a href="https://www.reddit.com/advertising/"&gt;If your content fits in an adv

In [112]:
subreddit_info = []
for child in subreddit_data['data']['children']:
    subreddit_info.append(child['data'])
subreddit_info = pd.DataFrame(subreddit_info)

In [113]:
subreddit_info[['user_is_muted',
       'user_can_flair_in_sr', 'display_name', 'header_img', 'title',
       'icon_img', 'display_name_prefixed', 'accounts_active',
       'public_traffic', 'subscribers', 'user_flair_richtext',
       'prediction_leaderboard_entry_type', 'emojis_enabled',
       'advertiser_category', 'public_description',
       'user_is_subscriber', 'disable_contributor_requests', 'allow_videogifs']]

,user_is_muted,user_can_flair_in_sr,display_name,header_img,title,icon_img,display_name_prefixed,accounts_active,public_traffic,subscribers,user_flair_richtext,prediction_leaderboard_entry_type,emojis_enabled,advertiser_category,public_description,user_is_subscriber,disable_contributor_requests,allow_videogifs
0,False,None,boston,https://b.thumbs.redditmedia.com/nUYNAt3U3uB57...,"Boston, MA",https://b.thumbs.redditmedia.com/0iaIAc8nva1Ck...,r/boston,None,False,431270,[],SUBREDDIT_HEADER,False,Local,"A reddit focused on the city of Boston, MA and...",True,False,True
1,False,None,newhaven,https://b.thumbs.redditmedia.com/KgLzaoX2o5g8w...,New Haven,,r/newhaven,None,False,24685,[],SUBREDDIT_HEADER,False,,Stuff about New Haven. For New Haven.,True,False,True
2,False,None,announcements,None,Announcements,https://b.thumbs.redditmedia.com/iTldIIlQVSoH6...,r/announcements,None,False,124905039,[],IN_FEED,False,Lifestyles,"Official announcements from Reddit, Inc.",True,True,True
3,False,None,wallstreetbets,https://b.thumbs.redditmedia.com/PrEHktHwx-B-c...,wallstreetbets,https://a.thumbs.redditmedia.com/nmh5l-zCsmmc3...,r/wallstreetbets,None,False,11257861,[],SUBREDDIT_HEADER,True,,Like 4chan found a Bloomberg terminal.,True,False,True
